In [189]:
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel

import spacy
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [200]:
# Load dataset
iphone_rev = pd.read_json('..\\scrape_reviews\\apple_reviews.json')
iphone_rev.head()

,name,stars,title,review
0,Akash Sinha,3,Worst battery,Worst battery performance.Iphone 11 is far bet...
1,Amazon Customer,4,Kidney as a load balancer,"Sold kidney bought this, now not feeling well ..."
2,S.Siva Ram Kris,5,"Another Fabulous IPhone ( IPhone 12, Blue 64GB)",The media could not be loaded. Another beauty...
3,Paras,5,The Beast,This was my first switch to an ios device afte...
4,Anupam,1,This phone is a joke and the joke is on us!!! ...,Extremely disappointed with this phone. It’s a...


In [191]:
iphone_rev['stars'].value_counts(dropna=False)
# positive reviews are higher

5    742
4    137
1    122
3     62
2     23
Name: stars, dtype: int64

In [192]:
reviews = iphone_rev['review'].to_list()
print(reviews[1])

Sold kidney bought this, now not feeling well but the number of days I am alive with one kidney will enjoy using this phone. Guys be careful if you rich it’s ok else sell something else but not kidney it hurts


In [193]:
# transforming words to its base form 
def lemmatization(texts, allowed_postags=['NOUN','VERB', 'ADJ', 'ADV']):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    final = [' '.join([token.lemma_ for token in nlp(text) if token.pos_ in allowed_postags]) for text in texts]
    return final

lemmatized_rev = lemmatization(reviews)
print(lemmatized_rev[1])

sell kidney buy now feel well number day alive kidney enjoy use phone guy careful rich ’ else sell else kidney hurt


In [194]:
# basic preprocessing to tokenize the reviews
processed_rev = [simple_preprocess(text) for text in lemmatized_rev]

print(processed_rev[1])

['sell', 'kidney', 'buy', 'now', 'feel', 'well', 'number', 'day', 'alive', 'kidney', 'enjoy', 'use', 'phone', 'guy', 'careful', 'rich', 'else', 'sell', 'else', 'kidney', 'hurt']


In [195]:
# including bigrams 
bigram_phrases = gensim.models.Phrases(processed_rev, min_count=5, threshold=1)
bigram = gensim.models.phrases.Phraser(bigram_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

texts = make_bigrams(processed_rev)

print(texts[1])

['sell', 'kidney', 'buy', 'now', 'feel', 'well', 'number', 'day', 'alive', 'kidney', 'enjoy', 'use_phone', 'guy', 'careful', 'rich', 'else', 'sell', 'else', 'kidney', 'hurt']


In [196]:
# tf-idf removal
# https://stackoverflow.com/questions/24688116/how-to-filter-out-words-with-low-tf-idf-in-a-corpus-with-gensim/35951190

id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value=0.03
words = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value<low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] 

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [197]:
# building model
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=8,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    random_state=42
)

In [198]:
# Visualization
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

C:\Users\Jobin\miniconda3\envs\nlp\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.341680 -0.152014       1        1  37.931346
4      0.136840  0.308803       2        1  30.508358
3      0.018426 -0.057545       3        1  13.926232
2     -0.117255 -0.043518       4        1   6.186117
0     -0.108620 -0.018712       5        1   4.693805
6     -0.110399 -0.021085       6        1   3.501357
1     -0.081773 -0.008468       7        1   1.643326
5     -0.078898 -0.007461       8        1   1.609460, topic_info=            Term        Freq       Total Category  logprob  loglift
306         good  291.000000  291.000000  Default  30.0000  30.0000
2            buy  175.000000  175.000000  Default  29.0000  29.0000
11       product  132.000000  132.000000  Default  28.0000  28.0000
1355  good_phone   46.000000   46.000000  Default  27.0000  27.0000
166          use  155.000000  155.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
1002        oppo    1.474699    2.255240   Topic8  -5.1372   3.7045
178         card    1.463044    2.256221   Topic8  -5.1451   3.6961
2029        vivo    1.263135    2.043916   Topic8  -5.2920   3.6480
1946      reboot    1.204981    1.996344   Topic8  -5.3392   3.6244
2483       frank    1.033241    1.819608   Topic8  -5.4929   3.5634

[326 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
931       5  0.906284      actually
422       6  0.842895    additional
498       3  0.949726         again
288       3  0.950444           all
425       2  0.995179          also
...     ...       ...           ...
1371      6  0.778171  worth_buying
414       1  0.214317          year
414       2  0.603984          year
414       4  0.136383          year
414       5  0.038967          year

[347 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 4, 3, 1, 7, 2, 6])

- From the above visualization we can see there are lot of reviews with the word `camera_quality`, which could indicate that the phone has some pretty great cameras
- Words like `performance` and `smooth` is also high which could suggest that this phone performes great
- Has great `build_quality` and `battery`
- There are also negative aspects such as `overpriced`, `waste_money`

In [199]:
# saving the model 
lda_model.save('model\\iphone_model.model')